# Creación del modelo

### Librerías

In [2]:
import pandas as pd
import json
import pyodbc

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import pickle
import joblib

### 1. Conexión a Base de Datos

In [3]:
id = open('credentials.json', 'r')
mi = json.load(id)

server = mi["server"] 
database = mi["database"] 
username = mi["user"]
password = mi["password"]

In [4]:
# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")

Error connecting to the database: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Unable to complete login process due to delay in login response (258)')


In [5]:
sales = pd.read_csv('sales.csv')
sales.head()

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,NaN,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01 00:00:00.000
1,2,False,Mr.,Keith,NaN,Harris,NaN,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01 00:00:00.000
2,3,False,Ms.,Donna,F.,Carreras,NaN,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01 00:00:00.000
3,4,False,Ms.,Janet,M.,Gates,NaN,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01 00:00:00.000
4,5,False,Mr.,Lucy,NaN,Harrington,NaN,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01 00:00:00.000


In [6]:
sales['Title'].value_counts()

Title
Mr.     490
Ms.     340
Sr.       6
Sra.      4
Name: count, dtype: int64

### Preprocesamiento

In [7]:
df_ohe = pd.get_dummies(sales[['Title', 'ModifiedDate']], columns=['Title'], prefix='T')
df_ohe.head()

,ModifiedDate,T_Mr.,T_Ms.,T_Sr.,T_Sra.
0,2005-08-01 00:00:00.000,True,False,False,False
1,2006-08-01 00:00:00.000,True,False,False,False
2,2005-09-01 00:00:00.000,False,True,False,False
3,2006-07-01 00:00:00.000,False,True,False,False
4,2006-09-01 00:00:00.000,True,False,False,False


In [8]:
# Convertir la columna 'fecha' a datetime
df_ohe['ModifiedDate'] = pd.to_datetime(df_ohe['ModifiedDate'])
# Obtener la fecha más antigua
fecha_min = df_ohe['ModifiedDate'].min()

# Calcular la diferencia en días respecto a la fecha mínima
df_ohe['nDay'] = (df_ohe['ModifiedDate'] - fecha_min).dt.days

# Eliminar columna de fecha original
df = df_ohe.drop('ModifiedDate', axis=1)
df.head()

,T_Mr.,T_Ms.,T_Sr.,T_Sra.,nDay
0,True,False,False,False,31
1,True,False,False,False,396
2,False,True,False,False,62
3,False,True,False,False,365
4,True,False,False,False,427


### Enrenamiento del modelo

In [9]:
# Definir variables independientes y variable dependiente
X = df.drop('nDay', axis=1)
y = df['nDay']

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R²): {r2}")


Error cuadrático medio (MSE): 102665.12674333289
Coeficiente de determinación (R²): -0.007124506304424738


### Empaquetar modelo

In [10]:
file = open('model.pkl', 'wb')
pickle.dump(model, file)
file.close()

In [11]:
# Desempaquetar para probar modelo

# Crear csv para probar modelo
rand_rows = [83, 23, 43, 98, 29]
#testAPI = sales.iloc[rand_rows, [2, 14]]
testAPI = X_train.iloc[rand_rows, :]
testAPI



,T_Mr.,T_Ms.,T_Sr.,T_Sra.
18,True,False,False,False
44,True,False,False,False
739,True,False,False,False
791,True,False,False,False
485,True,False,False,False


In [12]:
# Crear csv con datos de prueba
testAPI.to_csv('testAPI.csv', index=False)

In [13]:
# Cargar modelo
model = pickle.load(open('model.pkl','rb'))

# Cargar datos de prueba
df_testAPI = pd.read_csv('testAPI.csv')

# Predecir bankrupt
pred = model.predict(df_testAPI)
pred

array([366.53208556, 366.53208556, 366.53208556, 366.53208556,
       366.53208556])